In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# 生成一些二维数据
np.random.seed(42)
X = np.random.rand(100, 2)  # 100 个点，每个点有 2 维 (x, y)

# 训练 K-Means 模型，设定 K=3
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(X)

# 获取聚类中心
centroids = kmeans.cluster_centers_
labels = kmeans.labels_

# 可视化聚类结果
plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', alpha=0.6, edgecolors='k')
plt.scatter(centroids[:, 0], centroids[:, 1], c='red', marker='X', s=200, label="Centers")
plt.title("K-Means Clustering")
plt.legend()
plt.show()
